In [11]:
from lib.functions.get_temperature_path import get_temperatures_path

last_fetch_path = get_temperatures_path()
last_fetch_path

PosixPath('/home/naheridi/DataspellProjects/temperature-analysis/notebooks/../data/temperatures/2021-04-12T13:38:31')

In [12]:
from pandas import read_csv

temperatures = read_csv(last_fetch_path / '2_normalized' / 'filtered_dataset.csv', parse_dates=True, index_col=0)
temperatures

,wall_316,wall_210,bat_412_a,bat_316,bat_420,wall_412_a,bat_210,wall_420
2020-03-02 09:00:00,22.62,NaN,43.06,47.06,50.25,23.44,NaN,22.44
2020-03-02 09:10:00,22.62,24.25,43.19,46.81,49.88,23.50,43.50,22.44
2020-03-02 09:20:00,22.56,23.06,43.00,46.63,49.31,23.44,43.00,22.25
2020-03-02 09:30:00,22.62,21.94,42.69,46.56,49.31,23.50,42.06,22.69
2020-03-02 09:40:00,22.62,22.25,42.56,NaN,49.38,23.44,42.13,22.94
...,...,...,...,...,...,...,...,...
2021-12-04 12:50:00,19.94,24.81,23.00,41.37,46.81,21.44,40.00,22.56
2021-12-04 13:00:00,19.94,24.87,23.00,37.31,45.63,21.50,40.69,22.56
2021-12-04 13:10:00,20.06,24.87,23.06,39.94,49.31,21.56,39.88,22.62
2021-12-04 13:20:00,20.12,24.87,23.25,39.00,45.81,21.75,40.44,22.62


In [13]:
wall_temperatures = temperatures[[column for column in temperatures.columns if column.startswith('wall')]]
wall_temperatures = wall_temperatures.rename(columns={column: column.replace('wall', '').replace('_', '') for column in wall_temperatures.columns})
wall_temperatures_per_day = wall_temperatures.resample('D')

Считаем количество "выходов за границы допустимой температуры по ГОСТ". В качестве граничных значений выбраны наиболее "мягкие" границы значений 17-28°C.

In [14]:
from pandas import DataFrame

over_limit = DataFrame(
    [
        wall_temperatures_per_day.agg(lambda x: x[x < 17].count()).sum(axis=1),
        wall_temperatures_per_day.agg(lambda x: x[x > 28].count()).sum(axis=1)
    ],
    index=['< 17°C', '> 28°C']
).T
over_limit

,< 17°C,> 28°C
2020-03-02,0,2
2020-03-03,0,0
2020-03-04,0,0
2020-03-05,18,4
2020-03-06,0,0
...,...,...
2021-11-30,0,0
2021-12-01,0,0
2021-12-02,0,0
2021-12-03,0,0


In [15]:
analysis_path = last_fetch_path / '3_analysis'
analysis_path.mkdir(parents=True, exist_ok=True)

In [16]:
from plotly.express import bar

fig = bar(
    over_limit,
    title='Статистика выходов температуры помещений за рамки ГОСТа',
    labels={
        'index': 'Дата',
        'value': 'Кол-во замеров темп. вне ГОСТа',
        'variable': 'Граница ГОСТа'
    },
    template='seaborn'
)

fig.write_image(analysis_path / 'temperatures_over_limit.png', width=1080)
fig.show()

Исходя из полученных данных можно сделать вывод о том, что температура реже опускается ниже допустимой нормы, нежели превышает её.